# Agent build with LangChain agents and watsonx.data document library retrieval MCP server

#### Install the dependencies

In [ ]:
!pip install ibm-watsonxdata-dl-retrieval-mcp-server langchain-mcp-adapters langgraph langchain ibm_watson_machine_learning langchain_ibm

#### Add the environment variables

In [ ]:

import os

os.environ["WATSONX_API_KEY"]="<>"
os.environ["WATSONX_PROJECT_ID"]="<>"
os.environ["WATSONX_MODEL_ID"] = "meta-llama/llama-3-3-70b-instruct"
os.environ["WATSONX_URL"]="https://eu-de.ml.cloud.ibm.com"

#### Imports

In [8]:

from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from langchain_ibm import ChatWatsonx
import os
from pydantic import BaseModel, Field 
from typing import Optional

#### Setting up ChatWatsonx llm

In [9]:
parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 8000,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.7,
    }

model = ChatWatsonx(
    model_id=os.getenv("WATSONX_MODEL", "meta-llama/llama-3-3-70b-instruct"),
    apikey=os.getenv("WATSONX_API_KEY"),
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    url=os.getenv("WATSONX_URL", "https://eu-de.ml.cloud.ibm.com"),
    parameters=parameters
)


### MCP client creation (sse transport mode) 

In [ ]:
# Ensure that the ibm-watsonxdata-dl-retrieval-mcp-server MCP server is already running in port 8000 in the localhost with sse transport
client = MultiServerMCPClient(
    {
        "dl_retrieval": {
            "transport": "sse",
            "url": "http://localhost:8000/sse",
        },
    })

print("Created the mcp client in sse transport mode")

#### List the available tools

In [ ]:

async def printAvailableTools():  
    tools = []
    try:
        print("Attempting to create session...")
        async with client.session(server_name="dl_retrieval") as session:
            try:
                print("session created")
                mcp_tools = await load_mcp_tools(session)
                tools.extend(mcp_tools)

                print("\nAvailable Tools:")
                for i, tool in enumerate(tools, 1):
                    print(f"{i}. {tool.name}")
                    print(f"   Description: {tool.description}")
                    print("-" * 50)
            except Exception as e:
                    print(f"Failed to load MCP tools: {e}")
                    raise
    except Exception as e:
        print(f"Failed to create session: {e}")
        raise

await printAvailableTools()

#### Agent response structure

In [12]:
class AgentResponse(BaseModel):
    final_answer: Optional[str] = Field(description="Detailed answer for the initial User Query if the answer was found, else provide a clear explanation of why it could not be found and suggest possible next steps or alternatives.")

#### Creation of LangChain agent

In [13]:
async def createLangchainAgent(session):
    tools = []
    mcp_tools = await load_mcp_tools(session)
    tools.extend(mcp_tools)
    dl_agent = create_react_agent(
        model=model, 
        tools=tools,
        prompt=(
            "You are a helpful question answering agent\n"
            "Your task is to answer the user query by calling the relevant tools given to you\n"
        ),
        response_format=AgentResponse,
    )
    print("Agent created. Ready to interact!")
    return dl_agent

#### Agent invocation

In [14]:
async def callAgent(dl_agent,user_input):
    try:
        result = await dl_agent.ainvoke({
            "messages": [
                {
                    "role": "user",
                    "content": user_input
                }
            ]
        })
        return result
    except Exception as e:
        print(f"Error during invocation: {str(e)}")
        raise

#### Creates a LangChain session, initializes the agent, and processes a query asynchronously.

In [ ]:
user_input = "What is the highest invoice amount"
async def createAndRunAgent():
    try:
        print("Attempting to create session...")
        async with client.session(server_name="dl_retrieval") as session:
            print("session created")
            dl_agent = await createLangchainAgent(session)
            result = await callAgent(dl_agent,user_input)
            structured_resp = result.get('structured_response')
            if structured_resp and hasattr(structured_resp, 'final_answer'):
                print(structured_resp.final_answer)
            else:
                print(result)
    except Exception as e:
        print(f"Failed to create session: {e}")
        raise   
await createAndRunAgent()